Notebook de debug: foco en segmentos PF/AGRO/PJ, caída de principalidade (últimos 3 meses), sin inadimplencia,
análisis de caída de uso de tarjeta interno (últimos 3 meses, reducción >30%) y ejemplo JSON de contacto.

In [7]:
# imports y rutas
import os, sys, re
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
user = os.getlogin()

BASE_DIR = r"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira".format(USER=os.getlogin())
PRINCIPALIDADE_PATH = os.path.join(BASE_DIR, "indicador_nova_principalidade_historico.parquet")
EMI_CARTOES_PATH = os.path.join(BASE_DIR, "emi_compras_confirmacao_historico.parquet")  # nombre según su comentario
INAD_PATH = os.path.join(r"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet".format(USER=os.getlogin()), "base_inadimplentes.parquet")
SAIDA_DIR = os.path.join(BASE_DIR, "debug_output")
os.makedirs(SAIDA_DIR, exist_ok=True)

In [2]:
# Cargar datos principales (principalidade)
if not os.path.exists(PRINCIPALIDADE_PATH):
    raise FileNotFoundError(PRINCIPALIDADE_PATH)
principalidade = pd.read_parquet(PRINCIPALIDADE_PATH)

# Asegurar columnas clave existan
required = ['cpf_cnpj','ano_mes','pontos_principalidade','var_pontos','queda_flag','soma_quedas','sow_cartao','possui_cartao_credito']
missing = [c for c in required if c not in principalidade.columns]
if missing:
    # continuar pero creando columnas vacías para permitir flujo de debug
    for c in missing:
        principalidade[c] = np.nan

In [ ]:
# Cargar histórico de operaciones con tarjetas (emi_compras_confirmacao_historico)
# columnas esperadas: cpf_cnpj, marca, tpo_produto, dat_compra, funcionalidad, ano_mes, valor_compra (opcional)
if os.path.exists(EMI_CARTOES_PATH):
    emi = pd.read_parquet(EMI_CARTOES_PATH)
else:
    emi = pd.DataFrame(columns=['cpf_cnpj','marca','tpo_produto','dat_compra','funcionalidade','ano_mes','valor_compra'])

In [3]:
# Filtrar por segmentos que interesan y sin inadimplencia
segments_target = ['PF','AGRO','PJ']
principalidade['segmento'] = principalidade['segmento'].astype(str)
df = principalidade[principalidade['segmento'].isin(segments_target)].copy()

# Cargar base de inadimplentes si existe y excluir
if os.path.exists(INAD_PATH):
    inad = pd.read_parquet(INAD_PATH)
    cand_cols = [c for c in inad.columns if 'cpf' in c.lower() or 'doc' in c.lower()]
    if 'cpf_cnpj' not in inad.columns and cand_cols:
        inad = inad.rename(columns={cand_cols[0]:'cpf_cnpj'})
    inad['cpf_cnpj'] = inad['cpf_cnpj'].astype(str).str.replace(r'\D','',regex=True).str.lstrip('0')
    inad = inad[['cpf_cnpj']].drop_duplicates()
    df['cpf_cnpj'] = df['cpf_cnpj'].astype(str).str.replace(r'\D','',regex=True).str.lstrip('0')
    df = df.merge(inad.assign(flag_inad=1), on='cpf_cnpj', how='left')
    df = df[df['flag_inad'].isna()].copy()
else:
    df['flag_inad'] = np.nan

In [ ]:
# Mantener sólo registros con caída de principalidade en últimos 3 meses (soma_quedas >= 3)
# Asumimos que 'soma_quedas' ya está calculada en principalidade; si no, usar var_pontos en ventana.
df = df[df['soma_quedas'].fillna(0) >= 3].copy()

In [ ]:
# Analizar uso de tarjeta interno en últimos 3 meses: calcular gasto/volumen por cliente por mes y medir caída %
# Normalizar campos y filtrar por los meses relevantes presentes en df
meses_obj = sorted(df['ano_mes'].dropna().unique())[-3:]  # últimos 3 meses según principalidade
if not meses_obj:
    meses_obj = sorted(principalidade['ano_mes'].dropna().unique())[-3:]

emi['cpf_cnpj'] = emi['cpf_cnpj'].astype(str).str.replace(r'\D','',regex=True).str.lstrip('0')
emi['ano_mes'] = emi['ano_mes'].astype(str)

# Definir criterio de "tarjeta interna": ejemplo: marca contiene 'SICREDI' o tpo_produto indica 'TARJETA'
cond_interna = emi['marca'].astype(str).str.upper().fillna('').str.contains('SICREDI') | emi['tpo_produto'].astype(str).str.upper().fillna('').str.contains('TARJ')

emi_filtro = emi[emi['ano_mes'].isin(meses_obj) & cond_interna].copy()

# Agregar monto por cliente x mes (si no hay valor_compra se agrega conteo)
if 'valor_compra' in emi_filtro.columns and emi_filtro['valor_compra'].notna().any():
    agg = emi_filtro.groupby(['cpf_cnpj','ano_mes'])['valor_compra'].sum().reset_index()
else:
    agg = emi_filtro.groupby(['cpf_cnpj','ano_mes']).size().reset_index(name='valor_compra')

# Pivotar meses para tener columnas m1,m2,m3 donde m3 es el mes más reciente
agg_p = agg.pivot(index='cpf_cnpj', columns='ano_mes', values='valor_compra').fillna(0)
meses_sorted = sorted([m for m in agg_p.columns])
if len(meses_sorted) >= 3:
    m1, m2, m3 = meses_sorted[-3], meses_sorted[-2], meses_sorted[-1]
else:
    # completar si faltan meses
    m1 = meses_sorted[0] if meses_sorted else None
    m2 = meses_sorted[1] if len(meses_sorted)>1 else m1
    m3 = meses_sorted[-1] if meses_sorted else m1

agg_p = agg_p.rename(columns={m1:'m1', m2:'m2', m3:'m3'}) if m1 is not None else agg_p
agg_p = agg_p.reset_index().rename(columns={'index':'cpf_cnpj'}) if 'cpf_cnpj' not in agg_p.columns else agg_p.reset_index()
# Calcular promedio de periodo anterior y caída %
def calc_drop(row):
    try:
        prev = 0.0
        if (('m1' in row.index) and ('m2' in row.index)):
            prev = float((row.get('m1',0.0) + row.get('m2',0.0)) / (2 if pd.notnull(row.get('m2',None)) else 1))
        cur = float(row.get('m3',0.0))
        if prev == 0:
            return np.nan
        return (prev - cur) / prev
    except Exception:
        return np.nan

agg_p['card_drop_pct'] = agg_p.apply(calc_drop, axis=1)
# Filtrar caída mayor 30% (0.30)
agg_p['drop_flag_30'] = agg_p['card_drop_pct'] > 0.30

In [ ]:
# Unir con df para tener contexto y rankear por caída % (descendente)
agg_context = agg_p.merge(df[['cpf_cnpj','gestor','cod_agencia','sow_cartao']], on='cpf_cnpj', how='inner')
agg_context = agg_context.sort_values('card_drop_pct', ascending=False)
candidates_ranked = agg_context.copy()
# Guardar snapshot de candidatos
candidates_path = os.path.join(SAIDA_DIR, 'candidates_card_drop.parquet')
candidates_ranked.to_parquet(candidates_path, index=False)

In [ ]:
# Generar un ejemplo JSON de contacto para el primer candidato (si existe)
example = None
if not candidates_ranked.empty:
    r = candidates_ranked.iloc[0].to_dict()
    # Buscar datos de contacto en las fuentes disponibles (aquí se crea un ejemplo usando columnas esperadas)
    contato = {
        "titulo": f"CONTATO DO DIA: {r.get('cpf_cnpj','')} - {r.get('cod_agencia',0)}",
        "usuarioResponsavel": r.get('gestor',''),
        "usuarioSolicitante": r.get('gestor',''),
        "numeroAgencia": int(r.get('cod_agencia') or 0),
        "descricao": (
            f"Segmento: {r.get('tipo_pessoa','')} | Soma quedas: {r.get('soma_quedas','')}. "
            f"Drop cartão interno (pct): {r.get('card_drop_pct'):.2f} | Produtos básicos faltantes: (ver base). "
            "Ofertar: Conta Easy PF / Recuperar productos perdidos. "
            "Contactos: traer teléfonos y emails desde fuentes de contacto."
        ),
        "statusChamado": "A FAZER",
        "departamentoId": 0,
        "nomeDepartamento": None,
        "dataNecessidade": (datetime.today().date() + timedelta(days=15)).isoformat() + "T00:00:00.000Z",
        "categoriaChamado": "Giro de Carteira"
    }
    example = contato
# guardar ejemplo en JSON si existe
if example is not None:
    ex_path = os.path.join(SAIDA_DIR, 'example_contact.json')
    pd.Series([example]).to_json(ex_path, force_ascii=False, orient='records')

Siguientes pasos sugeridos:
- Validar nombres exactos de columnas en emi_compras_confirmacao_historico y ajustar cond_interna si la marca/tpo_produto usan otra nomenclatura.  
- Determinar las 3 agencias piloto y añadir filtro por cod_agencia.  
- Ajustar la regla de ranking para usar % caída del uso del cartão interno exactamente como se defina (ticket medio vs volumen).  
- Integrar campos de contacto (teléfonos/email) desde la base de associados para completar "descricao" con datos reales.

In [8]:
principalidade =  pd.read_parquet(rf"C:\Users\{user}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira")

ArrowInvalid: Could not open Parquet input source 'C:/Users/carola_luco/Sicredi/TimeBI_0730 - Documentos/01_Rotineiros/33_GiroCarteira/giro_de_carteira/bbmfiltros.py': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [ ]:
principalidade